<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2016-2017 Ivania Donoso - Antonio Ossa. Editado en 2018-2, 2019-1 y 2020-1 por Equipo Docente IIC2233.</font>
</p>

## Cómo recorrer o buscar en un grafo

Podemos recorrer un grafo usando los mismos métodos que utilizamos para árboles (BFS y DFS). Pero, debemos tener cuidado en la implementación.

Vamos a realizar estos recorridos sobre el grafo de amistades que ya hemos utilizado.

In [11]:
# Utilizaremos estas definiciones para Persona, Nodo y para Grafo, que ya habíamos revisado anteriormente.
class Persona:

    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def __repr__(self):
        return self.nombre

    
    
class Nodo:

    def __init__(self, valor):
        self.valor = valor
        
    def __repr__(self):
        return repr(self.valor)
    

    
class Grafo:

    def __init__(self, lista_adyacencia=None):
        self.lista_adyacencia = lista_adyacencia or {}

    def adyacentes(self, x, y):
        return y in self.lista_adyacencia[x]

    def vecinos(self, x):
        return self.lista_adyacencia[x]

    def agregar_vertice(self, x):
        self.lista_adyacencia[x] = set()

    def remover_vertice(self, x):
        self.lista_adyacencia.pop(x, None)
        for k, v in self.lista_adyacencia.items():
            if x in v:
                v.remove(x)

    def agregar_arista(self, x, y):
        if x in self.lista_adyacencia:
            self.lista_adyacencia[x].add(y)

    def remover_arista(self, x, y):
        vecinos_x = self.lista_adyacencia.get(x, set())
        if y in vecinos_x:
            vecinos_x.remove(y)

    def __repr__(self):
        texto_nodos = []
        for nodo, vecinos in self.lista_adyacencia.items():
            texto_nodos.append(f"Amigos de {nodo}: {vecinos}.")
        return "\n".join(texto_nodos)
    

## Y creamos algunos nodos, y un grafo de amistades
# Creamos a nuestros ayudantes y los guardamos en nodos.

bamavrakis = Nodo(Persona("Bastián", 15))
fvr1 = Nodo(Persona("Florencia V", 20))
aaossa = Nodo(Persona("Antonio", 21))
flobarrios = Nodo(Persona("Florencia B", 20))
mjjunemann = Nodo(Persona("Matías", 20))
fgvenegas = Nodo(Persona("Freddie", 10))
indonoso = Nodo(Persona("Ivania", 22))

# Definimos las amistades.

amistades = {
    bamavrakis: set([fvr1, aaossa, flobarrios, mjjunemann, fgvenegas, indonoso]),
    fvr1: set([flobarrios, fgvenegas, indonoso]),
    aaossa: set([fvr1, mjjunemann, indonoso]),
    mjjunemann: set([aaossa, fgvenegas]),
    flobarrios: set([fvr1, aaossa, mjjunemann, indonoso]),
    indonoso: set([fvr1, aaossa, flobarrios, fgvenegas]),
    fgvenegas: set([aaossa, bamavrakis])
}

grafo = Grafo(amistades)
grafo

Amigos de Bastián: {Ivania, Florencia B, Matías, Florencia V, Freddie, Antonio}.
Amigos de Florencia V: {Freddie, Florencia B, Ivania}.
Amigos de Antonio: {Ivania, Matías, Florencia V}.
Amigos de Matías: {Freddie, Antonio}.
Amigos de Florencia B: {Matías, Antonio, Ivania, Florencia V}.
Amigos de Ivania: {Florencia B, Antonio, Freddie, Florencia V}.
Amigos de Freddie: {Antonio, Bastián}.

Ahora intentaremos aplicar el mismo algoritmo de BFS que usamos para un árbol. Agregaremos un argumento `limite` para limitar la cantidad de nodos que visitamos:

In [12]:
from collections import deque


def bfs(grafo, inicio, limite = 20):   
    # La cola de siempre, comienza desde el nodo inicio.
    cola = deque([inicio])
    
    # Mientras queden vertices por visitar y no nos pasemos del limite de navegación
    while len(cola) > 0 and limite > 0:
        # Obtenemos de la cola el próximo vertice
        vertice = cola.popleft()
        print(vertice)
        # Agregamos los vecinos al stack
        for vecino in grafo[vertice]:
            cola.append(vecino)
        # Visitamos un nodo, bajamos el límite en 1
        limite -= 1
        

Ahora intentemos recorrer el grafo de amistades anterior, partiendo por Bastían:

In [13]:
bfs(amistades, bamavrakis)

Bastián
Ivania
Florencia B
Matías
Florencia V
Freddie
Antonio
Florencia B
Antonio
Freddie
Florencia V
Matías
Antonio
Ivania
Florencia V
Freddie
Antonio
Freddie
Florencia B
Ivania


Notamos rápidamente que lo impreso está repitiendo nombres. Si aumentamos `limite`, sigue pasando lo mismo. ¿Qué ocurre? 

Si vemos las amistades, eventualmente encontraremos que hay un ciclo de personas que son amigos uno tras el otro, volviendo a la persona inicial. Por ejemplo, Matías y Antonio son amigos mutuamente. Al mismo tiempo, Bastián tiene de amiga a Florencia B, quien tiene de amigo a Matías, quien tiene de amigo a Freddie, quien tiene de amigo a Bastián nuevamente. Luego, al visitar estos vértices, volvemos a agregar vértices que ya revisamos. Quedamos atrapados en un **ciclo** dentro del grafo.

¿Por qué no pasó esto con los árboles? Porque debido a la estructura de un árbol, solo hay **una única** forma de llegar a un vértice cualquiera, a través de su padre. Es decir, nunca se agrega dos veces un mismo vértice con el algoritmo anterior (para un árbol). Sin embargo, en el caso más general de los grafos, **debemos recordar qué vértices hemos visitado** hasta el momento, pues un grafo puede contener ciclos. Esto es importante para no quedarse atrapado en uno; en caso contrario, nuestro programa _nunca_ terminaría. De no agregar el límite en el ejemplo anterior, éste nunca terminaría.

A continuación, veremos las implementaciones para BFS y DFS que sí verifica no recorrer dos veces un mismo vértice.

### BFS

BFS recorre exhaustivamente el grafo, dado un punto de partida. Por lo tanto, si un nodo no fue visitado en el recorrido, significa que **no es alcanzable** desde ese punto de partida. 

Cabe mencionar que BFS no es un algorítmo recursivo. Dado un nodo de inicio, se exploran todos sus vecinos antes de explorar los vecinos de ellos. Por eso se llama _breadth_: de amplitud.

BFS es generalmente mejor que DFS para encontrar el camino más corto entre dos nodos (o cualquier camino).

In [16]:
from collections import deque

def bfs(grafo, inicio):
    # Vamos a mantener una lista con los nodos visitados.
    visitados = []
    # La cola de siempre, comienza desde el nodo inicio.
    queue = deque([inicio])
    
    while len(queue) > 0:
        vertice = queue.popleft()
        # Detalle clave: si ya visitamos el nodo, no hacemos nada!
        if vertice in visitados:
            continue

        # Lo visitamos
        print(vertice)
        visitados.append(vertice)
        # Agregamos los vecinos a la cola si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                queue.append(vecino)
    return visitados

In [17]:
bfs(amistades, bamavrakis)

Bastián
Ivania
Florencia B
Matías
Florencia V
Freddie
Antonio


[Bastián, Ivania, Florencia B, Matías, Florencia V, Freddie, Antonio]

Notamos que (1) el algoritmo termina gracias a que mantenemos un registro en `visitados` de todos los nodos revisados y (2) que obtenemos todas las personas como resultado, ya que todas son alcanzables desde alguna otra persona (grafo conexo). 

Pero si cambiamos un poco el grafo, dónde no todos los nodos son alcanzables desde otros nodos, solo obtenemos algunos:

In [19]:
grafo = {
    'A': ['B', 'C'],
    'B': ['D', 'A'],
    'C': ['D'],
    'D': ['B'],
    'E': ['F'],
    'F': ['E'],
    'G': ['A', 'F'],
}

In [20]:
print(f"Nodos alcanzables desde A: {bfs(grafo, 'A')}")

A
B
C
D
Nodos alcanzables desde A: ['A', 'B', 'C', 'D']


In [21]:
print(f"Nodos alcanzables desde E: {bfs(grafo, 'E')}")

E
F
Nodos alcanzables desde E: ['E', 'F']


In [22]:
print(f"Nodos alcanzables desde G: {bfs(grafo, 'G')}")

G
A
F
B
C
E
D
Nodos alcanzables desde G: ['G', 'A', 'F', 'B', 'C', 'E', 'D']


Haz el ejercicio de dibujar el grafo anterior, y ve como el algoritmo va recorriendo qué nodos. Como nuestra implementación usa una lista para guardar a los nodos visitados, se mantiene el orden en que fueron agregados. Revisa por qué tiene ese orden los nodos alcanzables de cada resultado.

### DFS

Al igual que BFS, el algoritmo DFS recorre exhaustivamente el grafo, dado un punto de partida. Si un nodo no fue visitado en el recorrido, significa que **no es alcanzable** desde ese punto de partida.

La diferencia con BFS es el orden en que se recorren los nodos. Como BFS es una búsqueda por nivel, encontrará los nodos según cuán lejos estén del punto de partida. Así, BFS garantiza que si se visitó al nodo $n_1$ antes que el nodo $n_2$, entonces $n_1$ está más cerca o a la misma distancia del punto de partida que $n_2$.

Por otro lado, DFS explora **en profundidad** cada uno de los vecinos de un nodo (por eso se llama _depth_), intentando llegar lo más lejos que puede antes de devolverse. En otras palabras, explora cada rama completa antes de cambiar a otra rama. Generalmente se prefiere DFS en vez de BFS cuando se quieren visitar todos los nodos del grafo. 

DFS sí es un algoritmo recursivo (a diferencia de BFS) por lo que tiene dos formas de implementación: iterativa y recursiva.

#### Iterativa

In [80]:
def dfs(grafo, inicio):
    # Vamos a mantener un set con los nodos visitados.
    visitados = set()

    # El stack de siempre, comienza desde el nodo inicio.
    stack = [inicio]

    while len(stack) > 0:
        vertice = stack.pop()
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vertice in visitados:
            continue

        # Lo visitamos
        print(vertice)
        visitados.add(vertice)

        # Agregamos los vecinos al stack si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                stack.append(vecino)

    return list(visitados)

In [81]:
dfs(amistades, bamavrakis)

Bastián
Antonio
Florencia V
Ivania
Freddie
Florencia B
Matías


[Florencia B, Bastián, Matías, Florencia V, Freddie, Antonio, Ivania]

In [82]:
print(f"Nodos alcanzables desde A: {dfs(grafo, 'A')}")

A
C
D
B
Nodos alcanzables desde A: ['B', 'A', 'D', 'C']


In [83]:
print(f"Nodos alcanzables desde E: {dfs(grafo, 'E')}")

E
F
Nodos alcanzables desde E: ['E', 'F']


In [84]:
print(f"Nodos alcanzables desde G: {dfs(grafo, 'G')}")

G
F
E
A
C
D
B
Nodos alcanzables desde G: ['E', 'A', 'B', 'F', 'G', 'C', 'D']


Nota que los resultados (como conjuntos de nodos) son iguales que en BFS, pero el orden es lo que cambia. Revisa en este caso el por qué aparecen en el orden en que imprimen.

#### Recursiva

In [85]:
# Vamos a mantener (como parámetro) un set con los nodos visitados.
def dfs_recursivo(grafo, vertice, visitados=None):
    visitados = visitados or set()

    # Lo visitamos
    print(vertice)
    visitados.add(vertice)

    for vecino in grafo[vertice]:
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vecino not in visitados:
            dfs_recursivo(grafo, vecino, visitados)

    return list(visitados)

In [86]:
dfs_recursivo(amistades, bamavrakis)

Bastián
Ivania
Florencia B
Matías
Freddie
Antonio
Florencia V


[Florencia B, Bastián, Matías, Florencia V, Freddie, Antonio, Ivania]

**Puedes poner en práctica implementación de algoritmos de recorrido o ponerlos en uso en los ejercicios propuestos 3.1,
 3.2, 3.3, 3.4 y 3.5.**